In [36]:
import pandas as pd
import folium
import json
import shapely
import geopandas as gpd

In [19]:
df = pd.read_csv("../data/northeast - northeast.csv").drop(columns=['Unnamed: 0', 'state', 'address', 'company', 'plant_access', 'false negative'])
df.head(5)

,geometry,index,true positive,false positive
0,POINT (-77.93776233164682 35.994855621300836),16714,0,1
1,POINT (-77.92529932224166 35.99343682817532),16715,0,1
2,POINT (-77.94270133967557 35.99150292717033),16716,0,1
3,POINT (-77.98518552248538 36.00260360302083),16723,1,0
4,POINT (-77.98553984793107 36.00216825681871),16724,1,0


In [20]:
color_dictionary = dict({0: 'green',
                   1: 'blue'})

label_dictionary = dict({'green': 'True Positive',
                         'blue': 'False Positive'})

In [23]:
df['color'] = df['true positive'].map(color_dictionary)
df['label'] = df['color'].map(label_dictionary)
df.head(5)

,geometry,index,true positive,false positive,color,label
0,POINT (-77.93776233164682 35.994855621300836),16714,0,1,green,True Positive
1,POINT (-77.92529932224166 35.99343682817532),16715,0,1,green,True Positive
2,POINT (-77.94270133967557 35.99150292717033),16716,0,1,green,True Positive
3,POINT (-77.98518552248538 36.00260360302083),16723,1,0,blue,False Positive
4,POINT (-77.98553984793107 36.00216825681871),16724,1,0,blue,False Positive


In [25]:
NC_LAT = 35.7596
NC_LNG = -79.0193

In [28]:
map = folium.Map(location=[NC_LAT, NC_LNG], zoom_start=8)
map

In [37]:
geo_df = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=df["geometry"].apply(shapely.wkt.loads))

In [42]:
folium.GeoJson(geo_df,
               icon=folium.Icon(color=['color']),
               tooltip=folium.features.GeoJsonTooltip(fields=['label'])
).add_to(map)

TypeError: unhashable type: 'list'

In [40]:
map